In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import io

In [0]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [0]:
file_train = files.upload()

Saving carvan_train.csv to carvan_train (3).csv


In [0]:
train = pd.read_csv(io.BytesIO(file_train['carvan_train.csv']))

In [0]:
file_test = files.upload()

Saving carvan_test.csv to carvan_test (3).csv


In [0]:
test = pd.read_csv(io.BytesIO(file_test['carvan_test.csv']))

In [0]:
test.shape

(4000, 85)

In [0]:
test['V86'] = None

In [0]:
train['data'] = "train"
test['data'] = "test"

In [0]:
all_data = pd.concat([train,test],axis=0,ignore_index=True)

In [0]:
length = all_data.shape[0]
length = length-1
length

9821

In [0]:
for i in range(0,length):
  if(all_data['V1'][i] == 32):
    all_data['V1'][i] = 5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for i in range(0,length):
  if(all_data['V6'][i] == 8):
    all_data['V6'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for i in range(0,length):
  if(all_data['V6'][i] == 9):
    all_data['V6'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#Creating Dummies

In [0]:
all_data['V1']=all_data['V1'].astype(np.object)
all_data['V4']=all_data['V4'].astype(np.object)
all_data['V5']=all_data['V5'].astype(np.object)
all_data['V6']=all_data['V6'].astype(np.object)

In [0]:
cat_vars = ['V1','V4','V5','V6']

In [0]:
for col in cat_vars:
  dummy=pd.get_dummies(all_data[col],drop_first=True,prefix=col)
  all_data=pd.concat([all_data,dummy],axis=1)
  del all_data[col]
  
del dummy  

In [0]:
final_train = all_data[all_data.data == 'train']
del final_train['data']
final_train.shape

(5822, 141)

In [0]:
final_test = all_data[all_data.data == 'test']
del final_test['data']
final_test.shape

(4000, 141)

In [0]:
final_train['V86'].value_counts()

0    5474
1     348
Name: V86, dtype: int64

In [0]:
from sklearn.utils import resample
 #Separate majority and minority classes
df_majority = final_train[final_train.V86==0]
df_minority = final_train[final_train.V86==1]

In [0]:
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=5474,    # to match majority class
                                 random_state=123) # reproducible results

In [0]:
df_majority['V86'].value_counts()

0    5474
Name: V86, dtype: int64

In [0]:
df_minority_upsampled['V86'].value_counts()

1    5474
Name: V86, dtype: int64

In [0]:
 #Combine majority class with upsampled minority class
final_train = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
final_train['V86'].value_counts()

1    5474
0    5474
Name: V86, dtype: int64

In [0]:
final_train['V86'] = final_train['V86'].astype(np.int)

In [0]:
final_train["V86"].head()

0    0
1    0
2    0
3    0
4    0
Name: V86, dtype: int64

In [0]:
df=final_train.drop('V86',axis=1)
y=final_train['V86']

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
#from imblearn.over_sampling import SMOTE

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.20,random_state = 10)

In [0]:
y_train.value_counts()

1    4411
0    4347
Name: V86, dtype: int64

In [0]:
y_test.value_counts()

0    1127
1    1063
Name: V86, dtype: int64

In [0]:
xgb_params = {  
                "learning_rate":[0.05],
                "gamma":[0.0],
                "max_depth": [8],
                "min_child_weight":[0.5],
                "max_delta_step":[3],
                "subsample":[1],
                "colsample_bytree":[0.2],
                "colsample_bylevel":[0.9],
                "reg_lambda":[0.1], 
                "reg_alpha":[0.1],
                "scale_pos_weight":[4],
                "n_estimators":[1200]
             }

In [0]:
xgb=XGBClassifier(objective='binary:logistic')

In [0]:
n_iter=1

random_search=RandomizedSearchCV(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=xgb_params)

In [0]:
random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0...
                   param_distributions={'colsample_bylevel': [0.9],
                                        'colsample_bytree': [0.2],
                                        'gamma': [0.0], 'learning_rate': [0.05],
      

In [0]:
report(random_search.cv_results_,3)

Model with rank: 1
Mean validation score: 0.992 (std: 0.00221)
Parameters: {'subsample': 1, 'scale_pos_weight': 4, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 1200, 'min_child_weight': 0.5, 'max_depth': 8, 'max_delta_step': 3, 'learning_rate': 0.05, 'gamma': 0.0, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.9}



In [0]:
test_prediction = random_search.predict(X_test)

In [0]:
confusion_matrix(y_test,test_prediction)

array([[1058,   69],
       [   0, 1063]])

In [0]:
roc_auc_score(y_test,test_prediction)

0.9693877551020409

In [0]:
#precision = TP/(TP+FP)
#recall = TP/(TP+FN) 
#fbeta = (5*precision*recall)/((4*precision)+recall)

# TP   FP
# FN   TN

precision = 1058/(1058+69)
recall = 1058/(1058+0)

In [0]:
fbeta = (5*precision*recall)/((4*precision)+recall)
fbeta

0.9871244635193133

In [0]:
#For train data

In [0]:
train_prediction = random_search.predict(X_train)

In [0]:
confusion_matrix(y_train,train_prediction)

array([[4289,   58],
       [   0, 4411]])

In [0]:
roc_auc_score(y_train,train_prediction)

0.9933287324591673

In [0]:
#precision = TP/(TP+FP)
#recall = TP/(TP+FN) 
#fbeta = (5*precision*recall)/((4*precision)+recall)

# TP   FP
# FN   TN

precision_train = 4289/(4289+58)
recall_train = 4289/(4289+0)

In [0]:
fbeta_train = (5*precision_train*recall_train)/((4*precision_train)+recall_train)
fbeta_train

0.9973027019485653

Test_Data

In [0]:
del final_test['V86']

In [0]:
test_data_predict = random_search.predict(final_test)

In [0]:
test_class_df = pd.DataFrame({'V86':test_data_predict})

In [0]:
test_class_df["V86"].value_counts()

0    3298
1     702
Name: V86, dtype: int64

In [0]:
test_class_df.to_csv('Project2_saketh_Reddy.csv',index=False)
files.download('Project2_saketh_Reddy.csv')

In [0]:
train["V86"].value_counts()

0    5474
1     348
Name: V86, dtype: int64